In [1]:
# Importing libraries
import pyarrow.parquet as pq
import pandas as pd

In [2]:
# Function to process the parquet files and return a single dataframe
def process_data(file_paths):
    dataframes = []
    for file_path in file_paths:
        table = pq.read_table(file_path)# Reading the parquet file as a table
        df = table.to_pandas()# Converting the table to a pandas dataframe
        dataframes.append(df)# Appending the dataframe to the list
    
    # Standardizing column names based on the first dataframe
    col_names_std = dataframes[0].columns.tolist()
    for df in dataframes:
        df.columns = col_names_std
    
    # Printing size of the dataframes
    for i, df in enumerate(dataframes):
        print(f"Shape of dataframe {i+1}: ", df.shape)
    print("Total Size of all dataframes: ", sum([df.shape[0] for df in dataframes]))

    # Concatenating the dataframes
    combined_df = pd.concat(dataframes, ignore_index=True)
    print("Shape of combined dataframe: ", combined_df.shape)
    
    return combined_df

In [3]:
# Reading the parquet files
file_paths = ['D:\TLC-NYC-Data-Analytics-End-To-End\env\Data\Monthly\y2023\yellow_tripdata_2023-01.parquet',
              'D:\TLC-NYC-Data-Analytics-End-To-End\env\Data\Monthly\y2023\yellow_tripdata_2023-02.parquet',
              'D:\TLC-NYC-Data-Analytics-End-To-End\env\Data\Monthly\y2023\yellow_tripdata_2023-03.parquet',
              'D:\TLC-NYC-Data-Analytics-End-To-End\env\Data\Monthly\y2023\yellow_tripdata_2023-04.parquet',
              'D:\TLC-NYC-Data-Analytics-End-To-End\env\Data\Monthly\y2023\yellow_tripdata_2023-05.parquet',
              'D:\TLC-NYC-Data-Analytics-End-To-End\env\Data\Monthly\y2023\yellow_tripdata_2023-06.parquet',
              'D:\TLC-NYC-Data-Analytics-End-To-End\env\Data\Monthly\y2023\yellow_tripdata_2023-07.parquet',
              ]
yellow_tripdata_2023 = process_data(file_paths)

Shape of dataframe 1:  (3066766, 19)
Shape of dataframe 2:  (2913955, 19)
Shape of dataframe 3:  (3403766, 19)
Shape of dataframe 4:  (3288250, 19)
Shape of dataframe 5:  (3513649, 19)
Shape of dataframe 6:  (3307234, 19)
Shape of dataframe 7:  (2907108, 19)
Total Size of all dataframes:  22400728
Shape of combined dataframe:  (22400728, 19)


In [4]:
# Checking for null values
nan_in_each_column = yellow_tripdata_2023.isna().sum()
print(nan_in_each_column)

VendorID                      0
tpep_pickup_datetime          0
tpep_dropoff_datetime         0
passenger_count          613638
trip_distance                 0
RatecodeID               613638
store_and_fwd_flag       613638
PULocationID                  0
DOLocationID                  0
payment_type                  0
fare_amount                   0
extra                         0
mta_tax                       0
tip_amount                    0
tolls_amount                  0
improvement_surcharge         0
total_amount                  0
congestion_surcharge     613638
airport_fee              613638
dtype: int64


In [5]:
# Filling null values
yellow_tripdata_2023['congestion_surcharge'] = yellow_tripdata_2023['congestion_surcharge'].fillna(0)
yellow_tripdata_2023['airport_fee'] = yellow_tripdata_2023['airport_fee'].fillna(0)
yellow_tripdata_2023['passenger_count'] = yellow_tripdata_2023['passenger_count'].fillna(0)
yellow_tripdata_2023['RatecodeID'] = yellow_tripdata_2023['RatecodeID'].fillna(1.0)
yellow_tripdata_2023['store_and_fwd_flag'] = yellow_tripdata_2023['store_and_fwd_flag'].fillna('N')

In [6]:
# Checking for null values again
nan_in_each_column = yellow_tripdata_2023.isna().sum()
print(nan_in_each_column)

VendorID                 0
tpep_pickup_datetime     0
tpep_dropoff_datetime    0
passenger_count          0
trip_distance            0
RatecodeID               0
store_and_fwd_flag       0
PULocationID             0
DOLocationID             0
payment_type             0
fare_amount              0
extra                    0
mta_tax                  0
tip_amount               0
tolls_amount             0
improvement_surcharge    0
total_amount             0
congestion_surcharge     0
airport_fee              0
dtype: int64


In [7]:
# Checking datatypes
yellow_tripdata_2023.dtypes

VendorID                          int64
tpep_pickup_datetime     datetime64[us]
tpep_dropoff_datetime    datetime64[us]
passenger_count                 float64
trip_distance                   float64
RatecodeID                      float64
store_and_fwd_flag               object
PULocationID                      int64
DOLocationID                      int64
payment_type                      int64
fare_amount                     float64
extra                           float64
mta_tax                         float64
tip_amount                      float64
tolls_amount                    float64
improvement_surcharge           float64
total_amount                    float64
congestion_surcharge            float64
airport_fee                     float64
dtype: object

In [8]:
# Converting datatypes
yellow_tripdata_2023['passenger_count'] = yellow_tripdata_2023['passenger_count'].astype('int64')
yellow_tripdata_2023['RatecodeID'] = yellow_tripdata_2023['RatecodeID'].astype('int64')
yellow_tripdata_2023['store_and_fwd_flag'] = yellow_tripdata_2023['store_and_fwd_flag'].astype(str)

In [9]:
# Checking datatypes again
yellow_tripdata_2023.dtypes

VendorID                          int64
tpep_pickup_datetime     datetime64[us]
tpep_dropoff_datetime    datetime64[us]
passenger_count                   int64
trip_distance                   float64
RatecodeID                        int64
store_and_fwd_flag               object
PULocationID                      int64
DOLocationID                      int64
payment_type                      int64
fare_amount                     float64
extra                           float64
mta_tax                         float64
tip_amount                      float64
tolls_amount                    float64
improvement_surcharge           float64
total_amount                    float64
congestion_surcharge            float64
airport_fee                     float64
dtype: object

In [10]:
# Checking for invalid values
mask1 = ~yellow_tripdata_2023['VendorID'].isin([1, 2]) # Check 'VendorID' column contains values other than 1 and 2
mask2 = ~yellow_tripdata_2023['RatecodeID'].isin([1, 2, 3, 4, 5, 6]) # Check 'RatecodeID' column contains values other than 1, 2, 3, 4, 5, and 6
mask3 = ~yellow_tripdata_2023['store_and_fwd_flag'].isin(['Y', 'N']) # Check if 'store_and_fwd_flag' column contains values other than 'Y' and 'N'
mask4 = ~yellow_tripdata_2023['payment_type'].isin([1, 2, 3, 4, 5, 6]) # Check if 'payment_type' column contains values other than 1, 2, 3, 4, 5, and 6
print(mask1.sum())
print(mask2.sum())
print(mask3.sum())
print(mask4.sum())

5820
110397
0
613638


In [11]:
# Dropping invalid values for VendorID
yellow_tripdata_2023 = yellow_tripdata_2023.drop(yellow_tripdata_2023[mask1].index)

In [12]:
# Replace the values in RatecodeID column with 1 if they are invalid
yellow_tripdata_2023.loc[mask2, 'RatecodeID'] = 1

In [13]:
# Replace the values in payment_type column with 5 if they are invalid
yellow_tripdata_2023.loc[mask4, 'payment_type'] = 5

In [14]:
# Checking for invalid values again
mask1 = ~yellow_tripdata_2023['VendorID'].isin([1, 2]) # Check 'VendorID' column contains values other than 1 and 2
mask2 = ~yellow_tripdata_2023['RatecodeID'].isin([1, 2, 3, 4, 5, 6]) # Check 'RatecodeID' column contains values other than 1, 2, 3, 4, 5, and 6
mask3 = ~yellow_tripdata_2023['store_and_fwd_flag'].isin(['Y', 'N']) # Check if 'store_and_fwd_flag' column contains values other than 'Y' and 'N'
mask4 = ~yellow_tripdata_2023['payment_type'].isin([1, 2, 3, 4, 5, 6]) # Check if 'payment_type' column contains values other than 1, 2, 3, 4, 5, and 6
print(mask1.sum())
print(mask2.sum())
print(mask3.sum())
print(mask4.sum())

0
0
0
0


In [15]:
# Total no. of rows after dropping invalid values
yellow_tripdata_2023.shape[0]

22394908

In [ ]:
# Shorting the dataframe removing some random data to reduce the size
print("Size before: ",yellow_tripdata_2023.shape[0])
n = 10594908
if n < len(yellow_tripdata_2023):
    drop_indices = yellow_tripdata_2023.sample(n, random_state=42).index
    df = yellow_tripdata_2023.drop(drop_indices)
else:
    print(f"Your DataFrame has only {len(yellow_tripdata_2023)} rows. Can't drop {n} rows.")
print("Size After: ",df.shape[0])

In [ ]:
# Dropping duplicates
print("Size before: ",yellow_tripdata_2023.shape[0])
yellow_tripdata_2023 = yellow_tripdata_2023.drop_duplicates().reset_index(drop=True)
print("Size After: ",yellow_tripdata_2023.shape[0])

In [ ]:
# Viewing info of the dataframe
yellow_tripdata_2023.info()

In [ ]:
# Viewing description of the dataframe
yellow_tripdata_2023.describe()

In [ ]:
# Viewing the dataframe after preprocessing
yellow_tripdata_2023.head(10)

In [ ]:
# Viewing the dataframe after preprocessing
yellow_tripdata_2023.tail(10)

In [ ]:
# Writing the dataframe to a parquet file
yellow_tripdata_2023.to_parquet('D:\TLC-NYC-Data-Analytics-End-To-End\env\Data\yellow_tripdata_2023.parquet.gzip', compression='gzip')